# This notebook is under active development ...

##  Objecte -Detection: sample Notebook using Azur SDK.NET

In [1]:
#r "nuget: Azure.ResourceManager.MachineLearning, 1.0.0-beta.1"
#r "nuget: Azure.Identity, 1.6.0"
#r "nuget: Azure.Security.KeyVault.Secrets, 4.3.0"
#r "nuget:Azure.Storage.Blobs, 12.12.0"
#r "Microsoft.VisualBasic"

#r "nuget: Microsoft.ML, 2.0.0-preview.22314.3"
#r "nuget: Microsoft.ML.AutoML, 0.20.0-preview.22314.3"
#r "nuget: Newtonsoft.Json, 13.0.1"

Installed Packages Azure.Identity, 1.6.0 Azure.ResourceManager.MachineLearning, 1.0.0-beta.1 Azure.Security.KeyVault.Secrets, 4.3.0 Azure.Storage.Blobs, 12.12.0 Microsoft.ML, 2.0.0-preview.22314.3 Microsoft.ML.AutoML, 0.20.0-preview.22314.3 Newtonsoft.Json, 13.0.1

In [1]:
// Import common usings.
using Azure.Storage.Blobs;
using Azure.Storage.Blobs.Models;
using System.IO;
using Microsoft.VisualBasic;
using Microsoft.VisualBasic.FileIO;
using Newtonsoft.Json;
using System.Runtime.Serialization;

### The following section define file paths, azure resources and other utilities to complete this machine learning opration in this notebook.

In [1]:
// Generate job id.
public string RandomString(int size, bool lowerCase)
        {
            StringBuilder builder = new StringBuilder();
            Random random = new Random();
            char ch;
            for (int i = 0; i < size; i++)
            {
                ch = Convert.ToChar(Convert.ToInt32(Math.Floor(26 * random.NextDouble() + 65)));
                builder.Append(ch);
            }
            if (lowerCase)
                return builder.ToString().ToLower();
            return builder.ToString();
        }


// File paths.
var vottFilePath= @"C:\Users\zehailem\Desktop\StopSignSeptember\Stopsept\StopSign-export.json";

var tempPath = Path.GetTempPath();
var guid = Guid.NewGuid().ToString();
var modelName="MlModel";
var jsonlFilePath = Path.Combine(tempPath, $"{guid}-{modelName}.AzureObjectDetectionTrainData.jsonl");

var MlTableFolder = Directory.CreateDirectory(Path.Combine(tempPath, "TrainMlTable"));
var mLTableFilePath = Path.Combine(MlTableFolder.FullName, "MLTable.");
 static string labelFilePath= @"Artifact/lables.json";
var onnxFilePath= @"Artifact/onnxmodel.onnx";
static string modelFilePath = @"Artifact/Model.zip";

// Azure resources Names.
string subscriptionName ="ML.NET Model Builder Demo";
string resourceGroupName = "ZewdPortlaExcercise";
string workspaceName ="BikeRentalWorkSpace";
string computeName ="AugustComp";
var experimentName = "ObjectDetectionNotebook";


The following sections define data contracts to  serilaize and diserilize values from vott generaged json. To learn more https://docs.microsoft.com/en-us/dotnet/machine-learning/how-to-guides/label-images-for-object-detection-using-vott

In [1]:
[DataContract]
public class AzureObjectDetectionLabel
    {
        [DataMember(Name ="label", IsRequired = true)]
        public string Label { get; set; }

        [DataMember(Name ="topX", IsRequired = true)]
        public float TopX { get; set; }

        [DataMember(Name = "topY", IsRequired = true)]
        public float TopY { get; set; }

        [DataMember(Name = "bottomX", IsRequired = true)]
        public float BottomX { get; set; }

        [DataMember(Name = "bottomY", IsRequired = true)]
        public float BottomY { get; set; }

        [DataMember(Name = "isCrowd", IsRequired = false)]
        public bool IsCrowd { get; set; }

    }

In [1]:
[DataContract]
public class AzureImageDetails
    {
        [DataMember(Name ="format", IsRequired = true)]
        public string Format { get; set; }

        [DataMember(Name = "width", IsRequired = true)]
        public string WidthPX
        {
            get => this.Width + "px";
        }

        [DataMember(Name = "height", IsRequired = true)]
        public string HeightPX
        {
            get => this.Height + "px";
        }

        public int Width { get; set; }

        public int Height { get; set; }
    }

In [1]:
[DataContract]
public class AzureObjectDetectionWithLabels
    {
        [DataMember(Name = "image_url", IsRequired = true)]
        public string ImageUrl { get; set; }

        [DataMember(Name = "image_details", IsRequired = true)]
        public AzureImageDetails ImageDetails { get; set; }

        [DataMember(Name = "label", IsRequired = true)]
        public List<AzureObjectDetectionLabel> Labels { get; set; } = new List<AzureObjectDetectionLabel>();
    }

In [1]:
[DataContract]
public class AzureObjectDetection
    {
        [DataMember(Name = "imageUrl", IsRequired = true)]
        public string ImageUrl { get; set; }

        [DataMember(Name = "imageDetails", IsRequired = true)]
        public AzureImageDetails ImageDetails { get; set; }

        [DataMember(Name = "label", IsRequired = true)]
        public AzureObjectDetectionLabel Label { get; set; }
    }

In [1]:
public class PointsItem
    {
        [JsonProperty("x")]
        public double X { get; set; }

        [JsonProperty("y")]
        public double Y { get; set; }
    }

In [1]:
public class BoundingBox
    {

        [JsonProperty("height")]
        public float Height { get; set; }

        [JsonProperty("width")]
        public float Width { get; set; }

        [JsonProperty("left")]
        public float Left { get; set; }

        [JsonProperty("top")]
        public float Top { get; set; }

        public string Label { get; set; }

        public override string ToString()
        {
            return $"Label: {this.Label}, Top: {this.Top}, Left: {this.Left}, Width: {this.Width}, Height: {this.Height}";
        }
    }

In [1]:

public class Size
    {
        [JsonProperty("width")]
        public int Width { get; set; }

        [JsonProperty("height")]
        public int Height { get; set; }
    }
public class Asset
    {

        [JsonProperty("format")]
        public string Format { get; set; }

        [JsonProperty("id")]
        public string Id { get; set; }

        [JsonProperty("name")]
        public string Name { get; set; }

        [JsonProperty("path")]
        public string Path { get; set; }

        [JsonProperty("size")]
        public Size Size { get; set; }

        [JsonProperty("state")]
        public int State { get; set; }

        [JsonProperty("type")]
        public int Type { get; set; }
    }

In [1]:
   
   
   public class RegionsItem
    {
        [JsonProperty("id")]
        public string Id { get; set; }

        [JsonProperty("type")]
        public string Type { get; set; }

        [JsonProperty("tags")]
        public List<string> Tags { get; set; }

        [JsonProperty("boundingBox")]
        public BoundingBox BoundingBox { get; set; }

        [JsonProperty("points")]
        public List<PointsItem> Points { get; set; }
    }

public class VideoSettings
    {
        [JsonProperty("frameExtractionRate")]
        public int FrameExtractionRate { get; set; }
    }


	public class Root
    {
        [JsonProperty("asset")]
        public Asset Asset { get; set; }

        [JsonProperty("regions")]
        public List<RegionsItem> Regions { get; set; }

        [JsonProperty("version")]
        public string Version { get; set; }
    }

	
        

	public class ActiveLearningSettings
    {
        [JsonProperty("autoDetect")]
        public string AutoDetect { get; set; }

        [JsonProperty("predictTag")]
        public string PredictTag { get; set; }

        [JsonProperty("modelPathType")]
        public string ModelPathType { get; set; }
    }

	public class TagsItem
    {
        [JsonProperty("name")]
        public string Name { get; set; }

        [JsonProperty("color")]
        public string Color { get; set; }
    }

public class VottJsonWrapper
    {
        [JsonProperty("name")]
        public string Name { get; set; }

        [JsonProperty("securityToken")]
        public string SecurityToken { get; set; }

        [JsonProperty("videoSettings")]
        public VideoSettings VideoSettings { get; set; }

        [JsonProperty("tags")]
        public List<TagsItem> Tags { get; set; }

        [JsonProperty("id")]
        public string ID { get; set; }

        [JsonProperty("activeLearningSettings")]
        public ActiveLearningSettings ActiveLearningSettings { get; set; }

        [JsonProperty("version")]
        public string Version { get; set; }

        [JsonProperty("lastVisitedAssetId")]
        public string LastVisitedAssetId { get; set; }

        [JsonProperty("assets")]
        public Dictionary<string, Root> Assets { get; set; }
    }

In [1]:
        public static List<AzureObjectDetection> GetImageDataFromVott(string filePath)
		{
			VottJsonWrapper values;
            using (StreamReader r = new StreamReader(filePath))
            {
                string json = r.ReadToEnd();
                values = JsonConvert.DeserializeObject<VottJsonWrapper>(json);
            }
			var imageData = new List<AzureObjectDetection>();
			foreach (KeyValuePair<string, Root> entry in values.Assets)
            {
                foreach (var region in entry.Value.Regions)
                {
                    foreach (var tag in region.Tags)
                    {
                        // add each file path to the list
                        var azureODImage = new AzureObjectDetection();
                        azureODImage.ImageUrl = entry.Value.Asset.Path;

                        var imageDetails = new AzureImageDetails()
                        {
                            Format = entry.Value.Asset.Format,
                            Width = entry.Value.Asset.Size.Width,
                            Height = entry.Value.Asset.Size.Height,
                        };

                        azureODImage.ImageDetails = imageDetails;

                        // values are normalized to size of image. between 0 and 1.
                        var imageLabel = new AzureObjectDetectionLabel()
                        {
                            Label = tag,
                            TopX = region.BoundingBox.Left / entry.Value.Asset.Size.Width,
                            TopY = region.BoundingBox.Top / entry.Value.Asset.Size.Height,
                            BottomX = (region.BoundingBox.Left + region.BoundingBox.Width) / entry.Value.Asset.Size.Width,
                            BottomY = (region.BoundingBox.Top + region.BoundingBox.Height) / entry.Value.Asset.Size.Height,
                        };

                        azureODImage.Label = imageLabel;
                        imageData.Add(azureODImage);
                    }
                }
            }
			return imageData;
		}

In [1]:
var dataImage= GetImageDataFromVott(vottFilePath);

## Azure storage setup

### Configure your storage connection string

In [1]:
string connectionString = Environment.GetEnvironmentVariable("AZURE_STORAGE_CONNECTION_STRING");
  
  //Create a BlobServiceClient object which will be used to create a container client
  BlobServiceClient blobServiceClient = new BlobServiceClient(connectionString);
  var allblobs = blobServiceClient.GetBlobContainers(BlobContainerTraits.Metadata, BlobContainerStates.System);

  // Get workspaceblobstore.
  var defaultBlob= allblobs.First(b=> b.Name.StartsWith("azureml-blobstore")).Name;
  var defaultContainer = blobServiceClient.GetBlobContainerClient(defaultBlob);

  // This is the storage where artifacts stored.
  var artifactBlob= allblobs.First(b=> b.Name.Equals("azureml")).Name;
  var artifactBlobContainer = blobServiceClient.GetBlobContainerClient(artifactBlob);

### Login into Azure in powershell

In [1]:
az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "id": "2f96ae42-240b-4228-bafa-26d8b7b03bf3",
    "isDefault": false,
    "managedByTenants": [],
    "name": "MSFT-CloudMS-CPT-PRD-01",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "zehailem@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "id": "8c4b5b03-3b24-4ed0-91f5-a703cd91b412",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Cosmos_C&E_Azure_AzureEngineeringSystems_100200",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "zehailem@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "id": "40dd91f6-ead1-40dd-a291-2ae37717981b",
    "isDefault": false,
  

### Use Azure SDKs to call into AzureML

In [1]:
using Azure.ResourceManager;
using Azure.ResourceManager.MachineLearning;
using System;
using System.Threading.Tasks;
using Azure.Core;
using Azure.ResourceManager;
using Azure.ResourceManager.Resources;
using Azure.Identity;
using Azure.ResourceManager.MachineLearning.Models;
using Azure;

In [1]:
var armClient = new ArmClient(new DefaultAzureCredential());
var subscription = armClient.GetSubscriptions();

### Select your subscription where you want to have machine learning oprations.

In [1]:
var selectedSubscription = subscription.Where(sub => sub.Data.DisplayName == subscriptionName).First();
var subId= selectedSubscription.Data.SubscriptionId;

### Select your resource group.

In [1]:
var resourceGroups = selectedSubscription.GetResourceGroups();
var rg= resourceGroups.Where(r=> r.Data.Name == resourceGroupName).First();

### Select your workspace.

In [1]:
MachineLearningWorkspaceResource selectedWorkspace = await rg.GetMachineLearningWorkspaceAsync(workspaceName);

### The following section generate jsonl file, mltable file and upload them with actual object images.

In [1]:
public void  CreateObjectDetectionJsonlFile(List<AzureObjectDetection> imageData, string imageFolder, string filePath)
        {
			var imageDataForRemote = imageData.Select(x =>
            {
                var imageLocalPath = new Uri(x.ImageUrl).LocalPath;
                var copy = new AzureObjectDetection();
                copy.ImageDetails = x.ImageDetails;
                copy.Label = x.Label;
                copy.ImageUrl = Path.Combine(imageFolder, Path.GetFileName(imageLocalPath));
                return copy;
            });

			 //To get the jsonl file format for new AutoML expectation https://docs.microsoft.com/en-us/azure/machine-learning/reference-automl-images-schema#object-detection.
            var imagesGroupByUrl = imageDataForRemote.GroupBy(i => i.ImageUrl);
            var imagesWithLables = new List<AzureObjectDetectionWithLabels>();
            foreach (var image in imagesGroupByUrl)
            {
                var imageWithUrl = new AzureObjectDetectionWithLabels();
                imageWithUrl.ImageUrl = $@"azureml://subscriptions/{subId}/resourcegroups/{rg.Data.Name}/workspaces/{selectedWorkspace.Data.Name}/datastores/workspaceblobstore/paths/{image.Key.Replace(@"\", "/")}";
                var images = image.Select(i => i);
                AzureImageDetails detail = null;
                foreach (var im in images)
                {
                    imageWithUrl.Labels.Add(im.Label);
                    if (detail == null)
                    {
                        detail = im.ImageDetails;
                    }
                }

                imageWithUrl.ImageDetails = detail;
                imagesWithLables.Add(imageWithUrl);
            }

			using (var fileWritter = new StreamWriter(new FileStream(filePath, FileMode.Create)))
            {
				foreach (var data in imagesWithLables)
                {
                    // flatten images
                    fileWritter.WriteLine(JsonConvert.SerializeObject(data));
                }
            }

		}

In [1]:
public void CreateMlTableFile(string selectedFolderName, string mlTableFilePath, string jsonlFilePath)
        {

            try
            {
                using (var outFile = File.CreateText(mlTableFilePath))
                {
                    var jsonlFileName = Path.GetFileName(jsonlFilePath);
                    var selectedFolder = selectedFolderName;
                    var mlTableContent = $@"paths:
  - file: azureml://datastores/workspaceblobstore/paths/{selectedFolder}/{jsonlFileName}
transformations:
  - read_json_lines:
        encoding: utf8
        invalid_lines: error
        include_path_column: false
  - convert_column_types:
      - columns: image_url
        column_type: stream_info";
                    outFile.Write(mlTableContent);
                }
            }
            catch (Exception e)
            {
                throw new Exception(e.Message);

            }
        }

In [1]:
 var imageFolder = new DirectoryInfo(Path.GetDirectoryName(vottFilePath)).Name;
CreateObjectDetectionJsonlFile(dataImage,imageFolder,jsonlFilePath);
CreateMlTableFile(imageFolder ,mLTableFilePath, jsonlFilePath);

### Upload images 

In [1]:
foreach(var data in dataImage)
{
	BlobClient blobClientImage = defaultContainer.GetBlobClient("./"+ imageFolder + "/" + Path.GetFileName(new Uri(data.ImageUrl).LocalPath));

	await blobClientImage.UploadAsync(new Uri(data.ImageUrl).LocalPath,true);
}

### Upload jsonl file

In [1]:
BlobClient blobClientImage = defaultContainer.GetBlobClient("./"+ imageFolder + "/" + Path.GetFileName(jsonlFilePath));

await blobClientImage.UploadAsync(jsonlFilePath,true);

### Upload MlTable

In [1]:
BlobClient blobClientImage = defaultContainer.GetBlobClient("./"+ imageFolder + "/TrainMlTable/" +  Path.GetFileName(mLTableFilePath));

await blobClientImage.UploadAsync(mLTableFilePath,true);

## Start Azure AutoML run.

In [1]:
public static async Task<MachineLearningJobResource> StartObjectDetectionAutoMlJobAsync(
    MachineLearningWorkspaceResource ws,
    string experimentName,
    string computeId,
    string selectedFolder, 
    string jobId)
{
    // Upload the MLTable in the default workspaceblobstore.
    var trainData = new MLTableJobInput(new Uri($"azureml://datastores/workspaceblobstore/paths/{selectedFolder}/TrainMlTable"))
    {
        Mode = InputDeliveryMode.ReadOnlyMount,
        Description = "Train data",
    };

    var trainingData = new TrainingDataSettings(trainData);
    ImageVerticalDataSettings dataSettings = new ImageVerticalDataSettings("label", trainingData);
    ImageLimitSettings limitSettings = new ImageLimitSettings()
    {
        Timeout = TimeSpan.FromHours(168)
    };

    AutoMLVertical taskDetails = new ImageObjectDetection(dataSettings, limitSettings)
    {
        LogVerbosity = LogVerbosity.Info,
        PrimaryMetric = ObjectDetectionPrimaryMetrics.MeanAveragePrecision,
    };

    var autoMLJob = new AutoMLJob(taskDetails)
    {
        ExperimentName = experimentName,
        DisplayName = "AutoMLJob ImageObjectDetection-" + Guid.NewGuid().ToString("n").Substring(0, 6),
        IsArchived = false,
        ComputeId = computeId,
        Resources = new ResourceConfiguration
        {
            InstanceCount = 3,
        },
        Properties = new Dictionary<string, string>
            {
                { "property-name", "property-value" },
            },
        Tags = new Dictionary<string, string>
            {
                { "tag-name", "tag-value" },
            },
        EnvironmentVariables = new Dictionary<string, string>()
            {
                { "env-var", "env-var-value" }
            },
        Description = "This is a description of test AutoMLJob for ImageObjectDetection from Ml.net Model Builder",
    };

    MachineLearningJobData MachineLearningJobData = new MachineLearningJobData(autoMLJob);
    ArmOperation<MachineLearningJobResource> jobOperation = await ws.GetMachineLearningJobs().CreateOrUpdateAsync(WaitUntil.Completed, jobId, MachineLearningJobData);
    MachineLearningJobResource jobResource = jobOperation.Value;
    return jobResource;
}

In [1]:
string jobId =RandomString(15, true);
var computeId = $"/subscriptions/{subId}/resourceGroups/{rg.Data.Name}/providers/Microsoft.MachineLearningServices/workspaces/{selectedWorkspace.Data.Name}/computes/{computeName}";
 var automlJob= StartObjectDetectionAutoMlJobAsync(selectedWorkspace, experimentName, computeId,imageFolder, jobId);